In [5]:
import pandas as pd
import rpy2.robjects as ro
import rpy2.rinterface as ri
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri
pandas2ri.activate()

In [2]:
%qtconsole

In [6]:
dat = pd.read_table('/home/jfear/sandbox/secim/data/QE2_mdw_8_NEG_Peak_List_PA_preprocess_20150507_log.tsv')
dat.set_index('rowID', inplace=True)

In [7]:
r = ro.r
rna = r['na.exclude']
rprincomp = r['princomp']
rsummary = r['summary']

In [9]:
res = rprincomp(rna(dat), cor=ri.TRUE)
summary = rsummary(res, loadings=ri.TRUE)

In [12]:
sdev = summary.rx2('sdev')
loadings = summary.rx2('loadings')

In [32]:
loadings_df = pandas2ri.ri2py_dataframe(loadings)

In [36]:
loadings_index = pandas2ri.ri2py_vector(loadings.names[0])
loadings_col = pandas2ri.ri2py_vector(loadings.names[1])

In [39]:
loadings_df.set_index(loadings_index)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
QE2_sbs_8_57.mzXML,-0.188739,0.300871,0.047588,0.204786,0.082230,0.140500,0.248571,0.203703,0.333151,0.232490,...,-0.142688,0.051470,0.043841,-0.017052,-0.276024,0.222329,-0.193726,0.205194,-0.062543,-0.116499
QE2_sbs_8_56.mzXML,-0.189659,0.306858,0.000048,0.190577,0.019157,0.004577,0.343774,0.121294,0.033617,0.304976,...,-0.000295,0.046406,-0.018289,0.152985,0.407319,-0.239047,0.198900,-0.455041,0.119455,0.204059
QE2_sbs_8_43.mzXML,-0.196058,-0.015688,-0.190686,-0.071781,-0.134014,0.110920,0.179334,-0.375678,0.211551,-0.351445,...,-0.255543,-0.153920,0.281744,-0.168611,0.232209,-0.007153,0.052148,-0.026379,-0.054061,0.005628
QE2_sbs_8_70.mzXML,-0.194687,0.007969,0.113939,-0.295397,0.223238,0.328182,-0.366639,0.000781,0.036144,0.408994,...,-0.106700,-0.146326,0.072941,0.178985,0.348596,-0.079384,-0.270308,0.228883,-0.103321,-0.097950
QE2_sbs_8_71.mzXML,-0.195296,0.127083,-0.165417,-0.066625,-0.169902,0.285488,-0.190407,-0.231000,0.106405,0.108614,...,0.157064,-0.295337,0.149020,-0.258974,-0.161445,0.080622,-0.044197,-0.388057,0.050326,-0.011255
QE2_sbs_8_48.mzXML,-0.197591,0.088464,-0.078517,-0.026730,-0.100910,0.188819,0.216392,-0.397943,-0.082943,0.003155,...,0.493062,0.085737,-0.082689,0.297666,-0.118389,-0.070785,-0.078863,0.280545,-0.012287,0.026659
QE2_sbs_8_64.mzXML,-0.182824,-0.363426,-0.080697,0.296709,0.244786,0.218549,0.128891,-0.073436,-0.077672,0.008439,...,-0.000069,-0.026885,-0.444950,-0.147415,0.003485,-0.254943,0.044190,0.027826,0.034744,-0.016513
QE2_sbs_8_49.mzXML,-0.196433,-0.079252,-0.210315,0.015638,-0.092376,-0.057545,0.186862,-0.201191,0.194148,0.167912,...,-0.356950,-0.040860,-0.155741,0.200995,-0.111633,0.178188,0.193431,0.194815,-0.002742,0.024084
QE2_sbs_8_44.mzXML,-0.198819,0.063780,-0.135749,-0.070335,-0.005596,-0.272264,-0.103620,0.005744,0.027419,-0.291244,...,0.011801,-0.095330,-0.472828,0.254021,0.273790,0.387128,-0.399613,-0.190979,-0.053358,0.087749
QE2_sbs_8_51.mzXML,-0.186790,-0.271811,0.199144,-0.374188,0.052244,0.209638,0.211852,0.207582,0.010223,-0.028229,...,0.084025,0.266782,-0.039836,-0.294221,0.152545,0.429364,0.236006,-0.019165,0.051943,-0.047093
